<a href="https://colab.research.google.com/github/21Ovi/TensorFlow_/blob/main/06_NPL_in_TensorFlow_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in TensorFlow

NPL has the goal of deriving information out of natural language (could be sequences text to speech).

Another common term for NLP problems is sequence to sequence problem (seq2seq)

In [2]:
!nvidia-smi

Wed Aug  2 16:19:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get helper functions

In [3]:
!wget https://raw.githubusercontent.com/21Ovi/Helper-Functions/main/helper_functions.py

# Import series of helper function for noteebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-08-02 16:19:57--  https://raw.githubusercontent.com/21Ovi/Helper-Functions/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10116 (9.9K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   9.88K  --.-KB/s    in 0s      

2023-08-02 16:19:57 (118 MB/s) - ‘helper_functions.py’ saved [10116/10116]



## Get a text dataset

The dataset we're going to using is Kaggle's introduction to NPL dataset (text samples of Tweets labelled as disaster or not disaster).

Resource : https://www.kaggle.com/competitions/nlp-getting-started

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# Unzip data
unzip_data("nlp_getting_started.zip")

--2023-08-02 16:20:02--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.128.128, 142.251.6.128, 172.217.214.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.128.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.007s  

2023-08-02 16:20:02 (88.4 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in.

But I prefer to get visualize straigt away.

So another way to do this is to use pandas.

In [5]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
# Suffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look loke?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # Create random indexes not higher than the total number of samples

for row in train_df[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text: \n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text: 
Pic of 16yr old PKK suicide bomber who detonated bomb in Turkey Army trench released http://t.co/pOL92mn8YZ

---

Target: 1 (real disaster)
Text: 
...//..// whao.. Pic of 16yr old PKK suicide bomber who detonated bomb in Turkey Army trench released http://t.co/tuVbR4lEP3

---

Target: 1 (real disaster)
Text: 
Look at the previous battles. Citizens were committing suicide so to not be under American control. The bomb was the only way. @NBCNews

---

Target: 0 (not real disaster)
Text: 
she's a suicide bomb

---

Target: 1 (real disaster)
Text: 
#Bestnaijamade: 16yr old PKK suicide bomber who detonated bomb in ... http://t.co/KSAwlYuX02 bestnaijamade bestnaijamade bestnaijamade beÛ_

---



### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
## Use train_train_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                             train_df_shuffled["target"].to_numpy(),
                                                                             test_size=0.1, # use 10% of training data for validation set
                                                                             random_state=42)

In [13]:
# Check lengths
len(train_sentences), len(train_labels),len(val_sentences), len(val_labels),

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a model is to convert your text to numbers.

There are few ways to do this, namely:
* `Tokenization`- direct mapping of token ( a token could be a workd or a character) to number.
* `Embedding` - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned)

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=max_vocab_length, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=max_length, # how long do you want your sequence to be
                                    pad_to_max_tokens=True)

In [17]:
len(train_sentences[0].split())

7

In [18]:
# Find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [19]:
# Setup text vactorize variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g how many words from a tweet does model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [20]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [21]:
# Create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street~ "
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)

In [23]:
print(f"Orignal text: \n {random_sentence}\
      n\nVectorized Version:")
text_vectorizer([random_sentence])

Orignal text: 
 Family mourns drowning of 'superhero' toddler with rare epilepsy: Bradley Diebold suffered hundreds of epilepticÛ_ http://t.co/unsayJDTu7      n
Vectorized Version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 302, 5088,  513,    6, 3372,  803,   14, 1783, 3024,    1,    1,
        3374,  482,    6,    1]])>

In [24]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() # get all of the unique words in our training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using Embedding Layer

The parameters we care most about for our embedding layer:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represnted by a vector 100 long
* `input_length` = length of the sequence being passed to the embedding layer

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                            output_dim=128, # output shape
                             embeddings_initializer="uniform",
                            input_length=max_length # how long is each input
                            )

embedding

In [26]:
# get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Orignal text: \n {random_sentence}\
      \n\nEmbedding Version: ")

# Embed the random sentence (turn it into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Orignal text: 
 AD Miles 'Hurricane of Fun : The Making of Wet Hot' https://t.co/SBZwRuwuFh      

Embedding Version: 


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.01310343, -0.00336312, -0.01755648, ...,  0.04002874,
          0.01191369, -0.02600776],
        [-0.01883497, -0.00054734, -0.01823536, ..., -0.02418811,
         -0.04633847, -0.03986847],
        [ 0.03578654,  0.00715614,  0.04779235, ...,  0.03199453,
         -0.01753502, -0.00118923],
        ...,
        [-0.03389204,  0.03927303, -0.02301474, ..., -0.01646081,
          0.04731871, -0.00566449],
        [-0.03389204,  0.03927303, -0.02301474, ..., -0.01646081,
          0.04731871, -0.00566449],
        [-0.03389204,  0.03927303, -0.02301474, ..., -0.01646081,
          0.04731871, -0.00566449]]], dtype=float32)>

In [27]:
# Checkout a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.01310343, -0.00336312, -0.01755648,  0.02454111,  0.0216077 ,
         0.03571569, -0.04484127, -0.04092791, -0.03073998,  0.02886033,
        -0.01254877,  0.04253453,  0.00054835, -0.04458416, -0.00626162,
        -0.03985675,  0.03773074, -0.03730911, -0.03818443, -0.00959686,
        -0.04039795,  0.043591  ,  0.04108217,  0.01096079, -0.04239252,
         0.04045964, -0.02365413, -0.02633631, -0.00699629, -0.04761539,
        -0.03431044, -0.00518075, -0.00976224, -0.03178819,  0.04746291,
         0.0385933 ,  0.02322655,  0.04788342, -0.0470666 , -0.00042095,
         0.00291525,  0.0411389 , -0.01306697,  0.00264863, -0.04282279,
         0.00443493, -0.00552577,  0.00279224,  0.02073656, -0.0469823 ,
         0.02899759,  0.02138833,  0.03114656, -0.03841072, -0.04991131,
         0.02703827,  0.01301504, -0.02497512,  0.00278122,  0.01355276,
        -0.04956226,  0.02833365, -0.04905144,  0.03735948,  0.00603535,
  

## Modelling a text dataset (running series of experiments)

No we've got way to turn our text sequences into numbers, it's time to start building a series of modelling experiments.

We've start with a baseline and move on from there

* `Model 0`: Naive Bayes (baseline), this is from SkLearn ML Map: https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
* `Model 1`: Feed-forward neural network (dense model)
* `Model 2`: LSTM Model (RNN)
* `Model 3`: GRU Model (RNN)
* `Model 4`: Bidirectional-LSTM Model (RNN)
* `Model 5`: 1D Convolutional Neural Network (CNN)
* `Model 6`: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
* `Model 7`: Same as model 6 with 10% of training data

How are we going to approch all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Fit a model
* Evaluate a model


### Model 0: Getting a baseline

as with all machine learning modelling experiments, it's important to create  a baseline model so you've got b benchmark for future eperiments to build a model

To create a baseline, we'll yse SkLearn's multinominal Naive Bayes using the TensorFlow TF-IDF formula to convert our words to numbers

> 🔑 **Note:** It's common practice to use non-DL algorithm as a baseline because of their speed and then later using DL to see if you can improve upon them.

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # convert words into numbers using tfidf
    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [30]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Create an evaluation function for our model experiments

We could evaluate all of our model's prediction with different metrics everytime, however, this will be cumbersome and could easily be fixed with a function

Let's create one to compare our model's prediction with truth labels using the following metrics:
* Accuracy
* Precision
* Recall
* F1-score

In [31]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1-score using "weightted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                   "recall": model_recall,
                   "F1": model_f1 }
  return model_results

In [32]:
# Get basedline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'F1': 0.7862189758049549}

### Model 1: A Simple Dense Model

In [33]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [47]:
# Build model with the Funtional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string) # input are 1-dimentional string
x = text_vectorizer(inputs) ## turn the input text into numbers
x = embedding(x) # create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x) # Create the output layer, want binary outputs so use sigmoid activation function

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [48]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [49]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [50]:
# Fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230802-163805
Epoch 1/5
215/215 [==============================] - 20s 65ms/step - loss: 0.6125 - accuracy: 0.6898 - val_loss: 0.5318 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4412 - accuracy: 0.8218 - val_loss: 0.4651 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3467 - accuracy: 0.8631 - val_loss: 0.4597 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.2845 - accuracy: 0.8908 - val_loss: 0.4610 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2376 - accuracy: 0.9117 - val_loss: 0.4743 - val_accuracy: 0.7848


In [52]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4743 - accuracy: 0.7848


[0.47426190972328186, 0.7847769260406494]

In [55]:
# Make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(762, 1)

In [56]:
# Look at the single prediction
model_1_pred_probs[0]

array([0.44749287], dtype=float32)

In [57]:
# Look at the first 10 predictions
model_1_pred_probs[:10]

array([[0.44749287],
       [0.7741495 ],
       [0.997948  ],
       [0.1144507 ],
       [0.1178981 ],
       [0.9328242 ],
       [0.93186617],
       [0.9920936 ],
       [0.97089046],
       [0.25539872]], dtype=float32)

In [60]:
# Convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [63]:
# Caculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.4776902887139,
 'precision': 0.7870511640590381,
 'recall': 0.7847769028871391,
 'F1': 0.7827022002767159}

In [64]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])